# MODELO 2º  CLUSTERIZACIÓN 

## Objetivo

#### Siguiendo el mismo objetivo que el primer modelo trataré de clusterizar los repartos de la base de datos inicial. 
En principio las librerías serán las mismas, aunque en este segundo modelo implementaré un apartado de **Visualización** con Geopandas y Matplotlib (posteriormente con TABLEAU) basado en las geocodificaciones de la API de Google. En el primer modelo no lleve a cabo esta visualización porque requería más recursos de Google al hacer llamadas a la API " geocoding" lo cual supone un coste.

Mencionar que la llamada a la **Distance Matrix** de Google saca por defecto las *geocodificaciones* aunque no las saca como output, motivo por el cual es imprescindible llamar posteriormente al servicio de Google "Geocoding" para poder visualizar basandonos en las latitudes y longitudes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math

#libreria googlemaps para python:
#! pip install -U googlemaps
import googlemaps

#Actualizo el pip instalador
#! pip install --upgrade pip

%matplotlib inline

### INICIO: LECTURA Y SELECCIÓN DE LOS DATOS

Los datos vienen en formato excel que es el que suelen usar en la empresa de transporte. La información que nos interesa es la relativa a la dirección (dirección,CP,Población) y fecha

In [ ]:
df_max = pd.read_excel('data_prueba.xls', usecols=[5, 6,7,15], convert_float=True, skip_footer=2, parse_dates=['FECHA REPARTO'])

###### Creo una columna unificando las 3 columnas relativas a dirección para mejor entendimiento de la API de Google

In [ ]:
df_max['DESTINO'] = df_max[['DIRECCION','CP DESTINO','POBLACION DESTINO']].apply(lambda x : '{}, {}, {}'.format(x[0],x[1],x[2]), axis=1)

In [ ]:
df_max.info()

In [ ]:
# FILTRADO POR FECHA DE REPARTO. En este caso el 04/02/2019
date_of_interest = pd.datetime(2019,2,4)# a solicitar al jefe de tráfico
df_max = df_max[ df_max['FECHA REPARTO']==date_of_interest]
df_max.shape

In [ ]:
# REMOVIDO DE ENTREGAS DUPLICADAS EN EL MISMO DESTINO, YA QUE LAS ENTREGAS SE SUELEN DUPLICAR POR EL Nº DE ENVÍOS, LO QUE 
# PARA NUESTRO CASO NO VAMOS A TENER EN CUENTA A LA HORA DE LA CLUSTERIZACIÓN
#AVISO DE DUPLICADOS EN LAS ENTREGAS:

same_directions = np.array([],dtype=str)#datos a sacar por pantalla al jefe de tráfico
differents_directions = np.array([],dtype=str)

for i in df_max['DESTINO']:            
        if i not in differents_directions:
                differents_directions = np.append(differents_directions,i)
        else:
            same_directions = np.append(same_directions,i)  
            #print('ten en cuenta que la dirección %s tiene más de una entrega' % same_direction)
            
# REMOVIDO DE ENTREGAS DUPLICADAS EN EL MISMO DESTINO PARA LIMPIAR EL DATAFRAME
df_max.drop_duplicates(inplace=True)
df_max.reset_index(drop=True,inplace=True)
len(df_max)

## GOOGLE MAPS PLATFORM

**LOGGIN**: obtengo una cuenta y un proyecto con Google Maps, lo cual me permitirá aprovechar sus utilidades. Hay que epsecificar los servicios que se requeriran ya que son llamadas de API diferenciadas

##### la API Key la guardo en un docuemtno .py la cual importo, aunque esto no me soluciona que se pueda usar por otros usuarios.

In [ ]:
from info_tfm import api_key

In [ ]:
user_google = googlemaps.Client(key=api_key)

#### Accedo a un REPOSITORIO abierto de desarrolladores de google maps para PYTHON, la cual me brinda la posibilidad de poder trabajar desde Python y acceder a los direferentes servicios Google Maps Platform tiene. 

Repositorio con la documentación: https://googlemaps.github.io/google-maps-services-python/docs/#   

# DISTANCE MATRIX 

#### El primer objetivo es crear la matriz de distancias entre los diferentes destinos de entrega. Google aporta una serie de servicios enre los que se incluye la llamada "distance matrix" (con una API especifica para ello). Esta es la que usaré a continuación.  En este servicio Google geocodifica por defecto para poder sacar los siguientes parámetros:

- Distancia
- Duración
- Duración en tráfico (si el transporte es automóvil)
- Status

In [ ]:
from googlemaps import distance_matrix

# Metodología de llamada a la API de Google 

**DIFERENCIA CON MODELO 1º**: como vimos en el Modelo 1º,  la API de Google no tolera meter más de 100 combinaciones de origenes-destinos (lo que equivale a 10 puntos de reparto), a la par que no tiene limite de peticiones diarias. Es por ello que llamábamos reiteradamente a la API de Google con 10 destinos y obteníamos el menor número de matrices de distancias (y con ello obteníamos los clusters de la clusterización  de 1º grado).

Condiciones Google:

- Limited to 100 elements per client-side request.
- Maximum of 25 origins and 25 destinations per server-side request.
- 1,000 server-side elements per second. *Note that the client-side service offers Unlimited elements per second, per project.

Para más información: https://developers.google.com/maps/documentation/javascript/distancematrix

#### PARÁMETROS DE LA DISTANCE MATRIX DE GOOGLE

Siguen siendo los mismo que el primer modelo:

- Mode=driving ---> porque nuestros repartos son en vehiculo en su mayor parte
- Departure_time = 'now'---> Debido a que la orden de repartos es en la mañana
- traffic_model = 'pessimistic'---> para no penalizar al conductor si la ruta sufre más atascos de lo normal
- region = '.es'---> usamos la ccTLD (códigos de pais) de España para prevenir duplicidades con direcciones de paises hispanos

### VARIABLE Nº DESTINOS

In [ ]:
# n tiene en cuenta el número de destinos a tomar en cuenta en el modelo. En mi caso cogeré 60 
# para ambos modelos por cuestiones económicas y de memoria
df= df_max.head(n=60)

In [ ]:
columna = df['DESTINO']

In [ ]:
len(columna)

## Solución al problema de la restricción  de GOOGLE

Como decíamos, en el primer modelo nos limitaba el problema de que no se podía sacar una matriz de distancias general para todos los puntos de reparto ya que no se podía superar los 10 destinos por limitación de Google, y por ello no se podían conectar todos los puntos de reparto entre ellos, quedando una matriz de distancias incompleta.

##### Metodología 2º Modelo: para solventar esta limitación y obtener una matriz de distancias total con el número de destinos que queramos procedo a iterar cada destino con el resto de destinos excepto con él mismo; guardando los resultados en un nueva array.

Es una doble iteración teniendo en cuenta las posiciones de cada destino para no coincidir consigo mismo y si con todos los destinos ajenos. Esta metodología resulta más simple y eficaz que el primer modelo a la par que más lenta y arriesgada.

In [ ]:
# LOOPS POR CADA ENTREGA

new_array_dm = np.array([],dtype=object)

for i in range(len(columna)):
    
    array_loop = np.arange(len(columna))
     
    for loop in array_loop:
        if loop!=i:
            
            dm = distance_matrix.distance_matrix(client=user_google, origins = columna[i],
                                destinations = columna[loop], mode = 'driving',
                                departure_time = 'now', traffic_model = 'pessimistic', 
                                region = '.es')
                      
            new_array_dm = np.append(new_array_dm,dm)            
           

In [ ]:
len(new_array_dm)

###### Tras revisión de errores compruebo que Google a veces (en una minima proporción) no devuelve duration_in_traffic por lo que no podemos continuar con esta variable, aunque será la variable idónea para nuestro propósito

## DESANIDAMIENTO DE LA INFORMACIÓN DE GOOGLE

En este punto tenemos una lista de clusters con el formato sacado directamente de Google. Ese formato presenta una estructura de listas y diccionarios superspuestos de donde tendré que sacar la información para crear mi Dataframe con un formato más homogéneo.

Google devuelve las siguientes Keys de donde parten los datos:

- origin_addresses
- destination_addresses
- rows: dentro viene la información relativa a tiempo y duración, así como duración de tráfico y status
- status

**NOTA**: CUANDO EL STATUS ES *NOT FOUND* NO HAY SALIDA DE LOS PARAMETROS, POR LO QUE NO COINCIDEN LAS LONGITUDES DE LAS SERIES QUE CONFORMAN EL DATAFRAME. POR ELLO CREO UN DATA FRAME INICIAL AL QUE QUITAMOS LAS LINEAS *NOT FOUND* PARA QUE LUEGO PUEDA PASAR LOS OTROS PARÁMETROS (DISTANCE, DURATION Y DURATION IN TRAFFIC) Y SE COLOQUEN EN EL ORDEN DESEADO
**NOTA**: como se observa---->  if values != 'Madrid, Spain': esto lo implemento porque mediante la observación veía que en algunas mínimas ocasiones daba problema la reducción que google hacía de las calles a esta expresión, duplicando destinos similares que darán problemas al futuro algoritmo de optimización de rutas

In [ ]:
origen = np.array([],dtype=str)
destino = np.array([],dtype=str)
distance = np.array([],dtype=int)
duration = np.array([],dtype=int)
#duration_in_traffic = np.array([],dtype=int)
status = np.array([],dtype=str)

for i in new_array_dm:
    for key,values in list(i.items()):  
        if values != 'Madrid, Spain':
            if key=='origin_addresses':                 
                    origen = np.append(origen,values)                
            if key=='destination_addresses':               
                    destino = np.append(destino,values)
            if key=='rows':
                for i in values:
                    for key,values in list(i.items()):
                        for i in values:                          
                            for key,values in list(i.items()):                                                          
                                if key=='distance':                                   
                                    for key,values in list(values.items()):
                                        if key=='value':
                                            distance = np.append(distance,values)
                                if key=='duration':
                                    for key,values in list(values.items()):
                                        if key=='value':
                                            duration = np.append(duration,values)
                                #if key=='duration_in_traffic':                                    
                                    #for key,values in list(values.items()):
                                        #if key=='value':
                                            #duration_in_traffic = np.append(duration_in_traffic,values)
                                if key=='status':
                                    status = np.append(status,values)                                

In [ ]:
len(origen),len(destino),len(status),len(distance),len(duration)#,len(duration_in_traffic)

In [ ]:
dfdata = { 'origen' : origen,
            'destino' : destino,       
             'status': status
              }
ddm = pd.DataFrame(dfdata,columns=['origen','destino','status'])
ddm = ddm[ ddm['status']!='NOT_FOUND']# Para evitar nulos y posibilitar que la extensión de las arrays coincidan 
ddm['distance'] = distance
ddm['duration'] = duration
#ddm['duration_in_traffic'] = duration_in_traffic  

In [ ]:
len(ddm)

In [ ]:
#ddm = ddm.filter(len(ddm['destino']) <= 20) #aplicar si da error el algoritmo de optimización ya que desetimaría las
#reducciones que Google hace de calles que no son "NOT FOUND" pero que tampoco encuentra su destino

In [ ]:
ddm.shape

**var_optimizer:** en este modelo posibilitaré cambiar de inicio la variable a optimizar. Se pretende que esta decisión la tome el jefe de tráfico en la visualización, junto con la fecha de reparto y Nº de trabajadores. Luego veremos que la elección de TABLEAU no fue idónea para poder hacer esto. 

In [ ]:
var_optimizer = 'duration'

In [ ]:
#ddm = ddm [(ddm[var_optimizer]!=0)] # No aplicar en este modelo
ddm.drop_duplicates(inplace=True)# lo pongo para ver si soluciona algo
ddm.reset_index(drop=True,inplace=True)

## OPTIMIZER: LINKAGE--->"SINGLE" (a kind linkage of self creeated for deliveries)

###### El siguiente paso es optimizar las rutas, en ete caso, ya no dentro de cada cluster de 1º Grado.

Como comenté en el 1º Modelo, tras probar AgglomerativeClustering y scipy.cluster.hierarchy tomo una de las decisiones más importantes de mi TFM y es la de intentar crear mi propio algoritmo de optimización y clusterización de las rutas en base a las variables que estimemos (duration, duration in traffic y distance).

In [ ]:
# Variables iniciales
    
index_origin = ddm['duration'].idxmin()
destination = ddm.loc[index_origin][0]
df_exit = pd.DataFrame(columns=('origen','destino','distance','duration','duration_in_traffic'))
df_exit_error = pd.DataFrame(columns=('origen','destino','distance','duration','duration_in_traffic'))

In [ ]:
while len(ddm)>0:
        try:
            index = ddm[ ddm['origen']==destination]['duration'].idxmin()            
        except ValueError:
            print('Google reduce alguna calle a "Madrid, Spain" y corta la cadena del index')
            
        df_exit = df_exit.append((ddm.loc[index]).T)
        origin = ddm.loc[index][0]
        destination = ddm.loc[index][1]
        ddm.drop(index,inplace=True) 
        ddm.drop(ddm  [ (ddm['origen']==destination) & (ddm['destino']==origin)].index,inplace=True)
        ddm.drop(ddm  [  ddm['destino']==destination].index,inplace=True)
        ddm.drop(ddm  [  ddm['destino']==origin].index,inplace=True)       


In [ ]:
df_exit

**Resultado**: con ello lo que hemos conseguido es una ruta optimizada conectando los destinos por la menor variable escogida. En nuestro caso, la duración.

Tras ello cogeré la array de la varible elegida para el proceso posterior de clusterización

In [ ]:
array_var = np.array(df_exit[var_optimizer])
array_var

# FASE CLUSTERIZACIÓN FINAL

En este momento disponemos de un DataFrame ordenado por la variable elegida (duration en nuestro caso) optimizadas anteriormente. La clusterización  Final consiste en reorganizar los clusters atendiendo al tiempo de reparto de los conductores. Es por ello que segmentaremos el dataframe en clusteres atendiendo a la variable **time_required_per_route**

Este criterio o variable limitará el número de destinos o rutas por cluster en función de que se cumpla el argumento que queramos. En este vaso, como en el primer modelo, queremos que se ajuste se ajuste al horario de los trabajadores

Esta fase es la clave para comprender la finalidad de nuestro modelo,  que tiene tanto una vertiente económica relacionada con la optimización del trabajo de cada repartidor, como decalidad del servicio otorgado

#### NOTA: otra var_optimizer a escoger es la *distancia*. Con ello conseguiramos un menor gasto en combustible para los repartos, lo que puede tener un impacto no solo económico sino medioambiental importante.

## CUMSUM Y CLUSTERIZACIÓN

### En este apartado procederé de dos manera:

- **1) Recomendaremos al jefe de tráfico el Nº de clusters óptimo para un tiempo de ruta dado (time_route)**
- **2) El jefe de tráfico dará el número de repartidores que dispone para calcular el Nº de clusters que se ajuste a su petición  y así poder tomar decisiones etratégicas**

### 1º)  RECOMENDACIÓN DE CLUSTERS: R_CLUSTERS

In [ ]:
# Definición: por defecto siempre dará 21600 segundos = 6 horas de reparto

def my_cumsum_func(column_duration,time_route=21600):
        grp = np.zeros(len(column_duration))
        grp[0] = 0
        #dfdata = { 'cumsum_duration' : column_duration,'clusters': grp }

        for i in range(1,len(column_duration)):

            if (column_duration[i-1] + column_duration[i]) <= time_route:
                grp[i] = grp[i-1]
                column_duration[i] = column_duration[i-1] + column_duration[i]
            else:
                grp[i] = grp[i-1] + 1
                
        result = np.array([column_duration,grp])

        return result

In [ ]:
time_required_per_route = 4000 # prueba con 1 hora aprox
array_var_cumsum = my_cumsum_func(array_var,time_required_per_route)

Completo el DataFrame con los resultados (el cumsum y el nº de cluster)

In [ ]:
df_exit['R_cumsum_var']=array_var_cumsum[0]
df_exit['R_clusters']=array_var_cumsum[1]

In [ ]:
df_exit

### 2º)  SOLICITUD CON Nº REPARTIDORES DADO: N_CLUSTERS

Como comenté anteriomente, trato de solicitar el nº de trabajadores usando el módulo visto en clase de *Flask*, pero no consigo obtener el entero y demás variables de manera sencilla, por lo que dejo el Nº trabajadores estático en 8. 

from flask import Flask

app= Flask("My work number")

@app.route('/ret_number/<int:n>', methods=['GET'])
def get_num(n):
    try:
        numb=int(n)
        return numb
    except:
        return "Could not find a number"
    
app.run()

In [ ]:
#JEFE DE TRÁFICO INDICA Nº DE TRABAJADORES:
N_works = 6

In [ ]:
array_var_2 = np.array(df_exit[var_optimizer])
array_var_2

In [ ]:
# Función diferenciada por la petición del Nº de trabajadores
def my_Ncumsum_func(column_duration,N_works):
    
        time_route_estimated = round(column_duration.sum()/N_works)
        
        grp = np.zeros(len(column_duration))
        grp[0] = 0

        for i in range(1,len(column_duration)):

            if (column_duration[i-1] + column_duration[i]) <= time_route_estimated:
                grp[i] = grp[i-1]
                column_duration[i] = column_duration[i-1] + column_duration[i]
            else:
                grp[i] = grp[i-1] + 1
                
        result = np.array([column_duration,grp])

        return result

In [ ]:
time_route_estimated_proof = round(array_var_2.sum()/N_works)
print(array_var_2.sum())
time_route_estimated_proof

In [ ]:
array_var_Ncumsum = my_Ncumsum_func(array_var_2,N_works)

In [ ]:
array_var_Ncumsum

In [ ]:
#Corrijo el defecto de los últimos repartos para que se incluyan en el cluster 
array_var_Ncumsum[1]

for i in range(len(array_var_Ncumsum[1])):
        if array_var_Ncumsum[1][i] > N_works:
            array_var_Ncumsum[1][i] = N_works
        

In [ ]:
array_var_Ncumsum

AUMENTO DEL DATAFRAME TOTAL CON LOS CLUSTERES Y EL CUMSUM

In [ ]:
df_exit['N_cumsum_var_']=array_var_Ncumsum[0]
df_exit['N_clusters']=array_var_Ncumsum[1]

In [ ]:
df_exit

# VISUALIZACIÓN 

##### El objetivo es obtener la longitud y latitud de los destinos. Para ello se podría hacer con GEOPANDAS, pero tiene limitación del número de peticiones ya que es necesario una KEY, por lo que accedo mediante la API que ya dispongo de GOOGLE

In [ ]:
from googlemaps import geocoding

In [ ]:
geocoding_array = np.array([],dtype=object)

for destino in range(len(df_exit)):
    
    df_exit_geocode = geocoding.geocode(client=user_google,address=df_exit.iloc[destino]['destino'])
    
    geocoding_array = np.append(geocoding_array,df_exit_geocode)
    

**Resultado:** este proceso es más corto que el de distance matrix pero en esencia similar. Me limito a obtener las localizaciones de Google Maps para los destinos dado y lo estructuro correctamente para pasar a mi Dataframe los valores de la latitud y de la longitud

In [ ]:
geocoding_array.shape

In [ ]:
lat = np.array([],dtype=float)
lng = np.array([],dtype=float)

for destino in geocoding_array:
         for key,values in list(destino.items()):
            if key=='geometry':
                for key,values in list(values.items()):
                    if key=='location':
                        for key,values in list(values.items()):
                            if key=='lat':
                                lat = np.append(lat,values)
                            if key=='lng':
                                lng = np.append(lng,values)
                            
lat,lng        

In [ ]:
df_exit['lng']=lng
df_exit['lat']=lat

Saco una tercera columna unificando lat-lng. Aunque a priori inecesario la dejamos por si es útil para la visualización en Tableau

In [ ]:
df_exit['lng-lat'] = df_exit[['lng','lat']].apply(lambda x : '{}, {}'.format(x[0],x[1]), axis=1)

In [ ]:
df_exit.reset_index(drop=True,inplace=True)

In [ ]:
df_exit

## ORDENACIÓN DE LAS RUTAS

El propósito ahora es indicar, con la creación de dos columnas nuevas (una para R_clusters y otra para N_clusters) el orden de reparto de las diferentes destinos dentro de cada cluster (de cada tipo, de ahí que se hagan dos columnas)

Para ello nos valdremos de una nueva función **my_order_route** que recoge cada cluster y ordena por cada observación. 

**Objetivo**: Esta ampliación del DataFrame es fundamental para facilitar la correcta visualización de las rutas en **Tableau** ya que así el jefe de tráfico pueda intuir de un vistazo el orden de reparto

In [ ]:
# Función orden de reparto:
def my_order_route(clusters_deliveries):  
     
        grp = np.zeros(len(clusters_deliveries),dtype=int)
        grp[0] = 0

        for i in range(1,len(clusters_deliveries)):

            if clusters_deliveries[i-1] == clusters_deliveries[i]:
                grp[i] = grp[i-1]+1
                
            else:
                grp[i] = 0
                
        result = np.array([grp])

        return result

1) Procedo con R_clusters

In [ ]:
# Creo la variable cogiendo la array de R_clsuters
array_var_2 = np.array(df_exit['R_clusters'])
# Procedo a pasarle la función de order_route
array_order_route = my_order_route(array_var_2)
# Paso la columna al DataFrame
df_exit['R_order_route'] = array_order_route[0]

2) Procedo con N_clusters

In [ ]:
array_var_3 = np.array(df_exit['N_clusters'])
array_N_order_route = my_order_route(array_var_3)
df_exit['N_order_route'] = array_N_order_route[0]

In [ ]:
df_exit

### GUARDO EL  DATA FRAME EN FORMATO EXCEL

Necesario para la visualización en Tableau

In [ ]:
final_document_excel = df_exit.to_excel('final_document.xls',sheet_name='df_exit')

## GRÁFICAS DE VISUALIZACIÓN

#### 1) Prueba visualización de la Recomendación dada al jefe de tráfico.  
Visualización con Scatter para ver la correspondiente agruapación de destinos por cluster y comprobar a simple vista si el modelo es idóneo

In [ ]:
# configurando el tamaño de la figura
fig, ax = plt.subplots(figsize=(7,7))
# le paso contenido 
plt.scatter(x = df_exit['lat'], y = df_exit['lng'], c=df_exit['R_clusters'], cmap='plasma', linewidths=2)
plt.xlabel('Latitud')
plt.ylabel('Longitud')
plt.suptitle('Clusters')

La finalidad de esta priemra visualización es conocer la estructura de distancias para observas posibles destinos alejados de las entregas urbanas. 

##### 2) Prueba visualización con el Nº de repartidores dado por el jefe de tráfico.  
Visualización con Scatter para ver la correspondiente agruapación de destinos por cluster y comprobar a simple vista si el modelo es idóneo

In [ ]:
#2) Con N trabajadores
#Visualización con Scatter para ver la correspondiente agruapación de destinos por cluster

# configurando el tamaño de la figura
fig, ax = plt.subplots(figsize=(7,7))
# le paso contenido 
plt.scatter(x = df_exit['lat'], y = df_exit['lng'], c=df_exit['N_clusters'], cmap='plasma', linewidths=2)
plt.xlabel('Latitud')
plt.ylabel('Longitud')
plt.suptitle('N_Clusters')


# GEOPANDAS

Para darle una visualización y tratamiento más idóneo al tipo de datos que usamos investigo acerca de GEOPANDAS, lo cual me perimite comprender mejor sobre el tratamiento GIS de mis datos

instalo GEOPANDAS
- ! pip install geopandas

En Windows da problemas al instalar geopandas (a diferencia de en linux). No obstante, no es necesario para nuestro modelo continuar con geopandas ya que sólo tiene finalidad didáctica

instalo DESCARTES: necesario para poder plotear un geopanda
- ! pip install descartes

In [ ]:
#! pip install geopandas

In [ ]:
#! pip install descartes

In [ ]:
import geopandas
import descartes

#### Trato de conseguir un archivo.shape de la Comunidad de Madrid por medio de la web "https://www.comunidad.madrid/servicios/mapas/geoportal-comunidad-madrid" sin éxito ya que no permiten su descarga

Procedo a contactar telefónicamente y me comentan que no está abierto al público esta información y habría que solictarlo si estudio en una Universidad Pública. Es por ello que trato de encontrar mapa en formato ".shape" sin éxito

#2. Installing Python Shapefile Library (PyShp)
! pip install pyshp: pero no voy a usar este sistema


The Python Shapefile Library (pyshp) provides read and write support for the Esri Shapefile format. The Shapefile format is a popular Geographic Information System vector data format created by Esri.

In [ ]:
#Es por ello que procedo a investigar si existe mapa de la Com.de Madrid en la base datasets de GEOPANDAS, 
#pero sólo está un punto de location de Madrid, por lo que no me sirve de base sobre el que proyectar mis 
#geolocalizaciones
gpd.datasets.available
path_geocode = gpd.datasets.get_path('naturalearth_cities')
df_geocode = gpd.read_file(path_geocode)
df_madrid = df_geocode [ df_geocode['name']=='Madrid']
df_madrid_geometry = df_madrid['geometry']
df_madrid_geometry.plot()

#### CREACIÓN GEODATAFRAME

In [ ]:
#Creating a GeoDataFrame from a DataFrame with coordinates
df_exit_geo_data = { 'City' : 'Madrid',
             'Country' : 'Spain',       
             'Latitude': df_exit['lng'],
             'Longitude': df_exit['lat'],
             'Clusters':df_exit['R_clusters']
              }

df_exit_geo = pd.DataFrame(df_exit_geo_data,columns=['City','Country','Latitude','Longitude','Clusters'])

In [ ]:
df_exit_geo

In [ ]:
#Covertimos el DataFRame en GeDataFrame para lo cual hay que especificar la geometría dados una longitud/latitud
#Estos puntos generados como geometrías son los que GeoPandas entenderá automáticamente como los puntos para plotear
gdf = gpd.GeoDataFrame(df_exit_geo, geometry = gpd.points_from_xy(df_exit_geo.Longitude, df_exit_geo.Latitude))

In [ ]:
gdf

In [ ]:
gdf.plot(color='red',c=gdf['Clusters'])
#fig, ax = plt.subplots(figsize=(5,5))  (opcional)
#gdf.plot(ax=ax)

In [ ]:
gpd.points_from_xy(x=gdf['Longitude'],y=gdf['Latitude'])

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
gdf.plot(ax=ax)

## RESULTADO FINAL

Tras la comlpicada obtención del documento .shape de la provincia de Madrid la visualización definitiva la finiquitaré en TABLEAU

#### df_exit: finalmente hemos conseguido un sólo documento con toda la información imprescindible para poder orientar al jefe de tráfico en la toma de decisiones y llevar a Tableau